In [2]:
#Importing the libraries
import numpy as np
import pandas as pd
from math import isnan
import ast
import re

In [3]:
#code for basic data preparation (This does not fully convert data in suitable required format)
def convert_data(input_string):
    modified_string = re.sub(r"defaultdict\(.*?\)", lambda x: f"'{x.group()}'", input_string)
    modified_string = modified_string.replace("'", '"') 
    modified_string = modified_string.replace("b'", '"')
    items = []
    buffer = ''
    open_brackets = 0

    for char in modified_string:
        if char in '([{':
            open_brackets += 1
        if char in ')]}':
            open_brackets -= 1
        if char == ',' and open_brackets == 0:
            items.append(buffer.strip())
            buffer = ''
        else:
            buffer += char
    if buffer:
        items.append(buffer.strip())
    
    converted_list = []
    for item in items:
        try:
            converted_list.append(ast.literal_eval(item))
        except (ValueError, SyntaxError):
            converted_list.append(item)

    return converted_list

# Malware

In [4]:
with open("CSV_malware.csv", 'r') as file:
    lines = file.readlines()
header = convert_data(lines[0])

In [5]:
column_order = header
column_order[0] = 'Country_Name'

In [6]:
processed_data = []
for line in lines[1:]:
    processed_row = convert_data(line)
    processed_data.append(processed_row)

In [7]:
list_length_39 = []
list_length_38 = []
list_other_lengths = []

for sublist in processed_data:
    if len(sublist) == 39:
        list_length_39.append(sublist)
    elif len(sublist) == 38:
        list_length_38.append(sublist)
    else:
        list_other_lengths.append(sublist)
        
len(list_other_lengths), len(list_length_39) , len(list_length_38)

(2235, 1593, 1173)

In [8]:
for i in range(len(list_length_39)):
    del list_length_39[i][14]

for sublist in list_length_39:
    assert len(sublist) == 38, f"List at index {list_length_39.index(sublist)} does not have length 38"

In [9]:
for sublist in list_other_lengths:
    if isinstance(sublist[-6], str) and isinstance(sublist[-7], str):
        sublist[-7] = sublist[-7] + sublist[-6]
        del sublist[-6]

list_other_lengths = [sublist for sublist in list_other_lengths if len(sublist) != 38 or list_length_38.append(sublist)]


In [10]:
len(list_other_lengths), len(list_length_39) , len(list_length_38)

(2235, 1593, 1173)

In [11]:
expected_types = [str, float, int, str, bytes, int, str, (str, list), int, int, str, str, int, str]
for sublist in list_other_lengths:
    if all(isinstance(sublist[i], expected_types[i]) for i in range(14)):
        del sublist[14]
list_other_lengths = [sublist for sublist in list_other_lengths if len(sublist) != 38 or list_length_38.append(sublist)]

In [12]:
len(list_other_lengths), len(list_length_39) , len(list_length_38)

(843, 1593, 2565)

In [13]:
expected_types = [str, str, int, str, bytes, int, str, (str, list), int, int, str, str, int, str]
for sublist in list_other_lengths:
    if all(isinstance(sublist[i], expected_types[i]) for i in range(14)):
        del sublist[14]
list_other_lengths = [sublist for sublist in list_other_lengths if len(sublist) != 38 or list_length_38.append(sublist)]

In [14]:
len(list_other_lengths), len(list_length_39) , len(list_length_38)

(516, 1593, 2892)

In [15]:
keywords = ['LLC Privacy ID# 14409419', 'S.A. DE C.V.', 'ltd.', 'Ltd', 'INC.', 'LLC Privacy ID# 12761418',
    'LLC Privacy ID# 7259l17', 'Ltd.', 'LTD.', 'LLC Privacy ID# 841802', 'S.L.',
    'LLC (PrivacyProtect.org)', 'Corp', 'LLC', 'Inc.','Inc - This domain might be for sale on GoDaddy.com or Sedo.com','Lda']

In [16]:
for sublist in list_other_lengths:
    if len(sublist) > 20:
        if 'd' in sublist[20][1]:
            if sublist[19] in keywords:
                sublist[18] = str(sublist[18]) + str(sublist[19])
                del sublist[19]
list_other_lengths = [sublist for sublist in list_other_lengths if len(sublist) != 38 or list_length_38.append(sublist)]

In [17]:
expected_types = [str, float, int, str, bytes, int, int, (str, list), int, int, str, str, int, str]
for sublist in list_other_lengths:
    if all(isinstance(sublist[i], expected_types[i]) for i in range(14)):
        del sublist[14]
list_other_lengths = [sublist for sublist in list_other_lengths if len(sublist) != 38 or list_length_38.append(sublist)]

In [18]:
expected_types = [str, float, int, str, bytes, int, int, (str, list), int, int, str, str, int, str]
for sublist in list_other_lengths:
    del sublist[14]
list_other_lengths = [sublist for sublist in list_other_lengths if len(sublist) != 38 or list_length_38.append(sublist)]

In [19]:
len(list_other_lengths), len(list_length_39) , len(list_length_38)


(21, 1593, 3387)

In [20]:
combined_list = list_length_39 + list_length_38
df_malware = pd.DataFrame(combined_list)
header[0] = 'Country_Name'
df_malware.columns = header
df_malware.replace('nan', np.nan, inplace=True)
df_malware = df_malware[column_order]

In [21]:
df_malware.shape

(4980, 38)

In [22]:
df_malware.head()

,Country_Name,ASN,TTL,IP,Domain,oc_8,longest_word,Domain_Name,dec_32,subdomain,...,len,typos,tld,shortened,Registrar,obfuscate_at_sign,Alexa_Rank,3gram,numeric_percentage,puny_coded
0,US,46606.0,14399,70.40.220.126,b'dionneg.com.',0,dio,DIONNEG.COM,0,0,...,8,"[(g.co, 90), (qq.com, 86)]",com,-1.0,FastDomain Inc.,0,-1.0,"[dio, ion, onn, nne, neg]",0.000000,0
1,US,16509.0,3599,15.164.151.80,b'vipprojects.cn.',0,projects,vipprojects.cn,0,0,...,12,"[(4.cn, 86), (360.cn, 86)]",cn,-1.0,杭州名商网络有限公司,0,-1.0,"[vip, ipp, ppr, pro, roj, oje, jec, ect, cts]",0.000000,0
2,US,395954.0,599,23.110.187.67,b'hhj3.cn.',0,hh,hhj3.cn,0,0,...,5,"[(miitbeian.gov.cn, 86), (miibeian.gov.cn, 86)]",cn,-1.0,成都西维数码科技有限公司,0,-1.0,"[hhj, hj3]",11.111111,0
3,US,395954.0,599,23.110.187.67,b'hhj3.cn.',0,hh,hhj3.cn,0,0,...,5,"[(miitbeian.gov.cn, 86), (miibeian.gov.cn, 86)]",cn,-1.0,成都西维数码科技有限公司,0,-1.0,"[hhj, hj3]",11.111111,0
4,SC,136800.0,599,154.83.190.100,b'hryspap.cn.',0,pap,hryspap.cn,0,0,...,8,"[(miitbeian.gov.cn, 86), (miibeian.gov.cn, 86)]",cn,-1.0,厦门商中在线科技股份有限公司（原厦门商中在线科技有限公司）,0,-1.0,"[hry, rys, ysp, spa, pap]",0.000000,0


In [23]:
bytes_rows_index = df_malware[df_malware['TTL'].apply(lambda x: isinstance(x, bytes))].index


In [24]:
for i in bytes_rows_index:
    df_malware.at[i, 'IP'], df_malware.at[i, 'Domain'] = df_malware.at[i, 'Domain'], df_malware.at[i, 'IP']
    df_malware.at[i, 'Domain'], df_malware.at[i, 'Country_Name'] = df_malware.at[i, 'Country_Name'], df_malware.at[i, 'Domain']
    df_malware.at[i, 'Domain'], df_malware.at[i, 'TTL'] = df_malware.at[i, 'TTL'], df_malware.at[i, 'Domain']

In [25]:
df_malware['TTL'] = df_malware['TTL'].astype(int)

In [26]:
df_malware= df_malware[df_malware['hex_32'].apply(lambda x: isinstance(x, int))]

In [27]:
df_malware['hex_32'] = df_malware['hex_32'].astype(int)

In [28]:
df_malware['Domain_Age'] = df_malware['Domain_Age'].str.replace(' days', '')
df_malware['Domain_Age'] = pd.to_numeric(df_malware['Domain_Age'], errors='coerce').astype('Int64')

In [29]:
ind = [4979,4974,4973,4960,4959,4958]
df_malware = df_malware.drop(index=ind)

df_malware = df_malware.reset_index(drop=True)


In [30]:
df_malware['Name_Server_Count'] = df_malware['Name_Server_Count'].astype(float)

In [31]:
df_malware.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4971 entries, 0 to 4970
Data columns (total 38 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Country_Name        4032 non-null   object 
 1   ASN                 4034 non-null   float64
 2   TTL                 4971 non-null   int64  
 3   IP                  4077 non-null   object 
 4   Domain              4971 non-null   object 
 5   oc_8                4971 non-null   int64  
 6   longest_word        4971 non-null   object 
 7   Domain_Name         3939 non-null   object 
 8   dec_32              4971 non-null   int64  
 9   subdomain           4971 non-null   int64  
 10  sld                 4971 non-null   object 
 11  Registrant_Name     61 non-null     object 
 12  hex_32              4971 non-null   int64  
 13  Domain_Age          3797 non-null   Int64  
 14  entropy             4971 non-null   float64
 15  Name_Server_Count   4399 non-null   float64
 16  Emails

In [32]:
df_malware.head()

,Country_Name,ASN,TTL,IP,Domain,oc_8,longest_word,Domain_Name,dec_32,subdomain,...,len,typos,tld,shortened,Registrar,obfuscate_at_sign,Alexa_Rank,3gram,numeric_percentage,puny_coded
0,US,46606.0,14399,70.40.220.126,b'dionneg.com.',0,dio,DIONNEG.COM,0,0,...,8,"[(g.co, 90), (qq.com, 86)]",com,-1.0,FastDomain Inc.,0,-1.0,"[dio, ion, onn, nne, neg]",0.000000,0
1,US,16509.0,3599,15.164.151.80,b'vipprojects.cn.',0,projects,vipprojects.cn,0,0,...,12,"[(4.cn, 86), (360.cn, 86)]",cn,-1.0,杭州名商网络有限公司,0,-1.0,"[vip, ipp, ppr, pro, roj, oje, jec, ect, cts]",0.000000,0
2,US,395954.0,599,23.110.187.67,b'hhj3.cn.',0,hh,hhj3.cn,0,0,...,5,"[(miitbeian.gov.cn, 86), (miibeian.gov.cn, 86)]",cn,-1.0,成都西维数码科技有限公司,0,-1.0,"[hhj, hj3]",11.111111,0
3,US,395954.0,599,23.110.187.67,b'hhj3.cn.',0,hh,hhj3.cn,0,0,...,5,"[(miitbeian.gov.cn, 86), (miibeian.gov.cn, 86)]",cn,-1.0,成都西维数码科技有限公司,0,-1.0,"[hhj, hj3]",11.111111,0
4,SC,136800.0,599,154.83.190.100,b'hryspap.cn.',0,pap,hryspap.cn,0,0,...,8,"[(miitbeian.gov.cn, 86), (miibeian.gov.cn, 86)]",cn,-1.0,厦门商中在线科技股份有限公司（原厦门商中在线科技有限公司）,0,-1.0,"[hry, rys, ysp, spa, pap]",0.000000,0


In [33]:
df_malware.to_csv('modified_malware.csv',index=False)

# Benign

In [34]:
key = [29465,65232,173216, 265778, 265779,318686, 318687, 318688,173215, 265775, 265776,318681, 318682,39591,47147,47148,51716,59436,65174,67680,67681,68806,68807,78975,81201,93102,93103,113693,114177,114178,132166,137032,137033,154187,154188,154189,175442,208857,230268,238977,238978,238979,238980,269215,295124,312569,312570,323129,356733,356734,356735,424862,424863,462188,462189,469882,469883,481203,481204,481561,481562,485302,485303,496243,496244,
      39590,173191, 265742, 265743,318642, 318643,47145,51712,59431,65168,67672,67673,68796,68797,78964,81189,93088,93089,113678,114160,114161,132148,
 137012, 173169, 265712, 265713,318606, 318607,137013,154165,154166,175417,208831,230241,238947,238948,238949,269181,295089,312532,312533,
 323089,356690,356691,356692,424817,424818,462141,462142,469833,469834,481152,481153,481508,481509,485247,485248,496186,496187,39589,47143,
 51709,59427,65163,67665,67666,68787,68788,78954,81178,93075,93076,113664,114144,114145,132131,136993,136994,154144,154145,175394,208807,230216,
 238919,238920,238921,269150,295057,312498,312499,323052,356650,356651,356652,424775,424776,462097,462098,469787,469788,481104,481105,
 481458,481459,485195,485196,496132,496133]

In [35]:
with open("CSV_benign.csv", 'r') as file:
    lines = file.readlines()
cnt = 0;
processed_data = []

for line in lines[1:]:
    if cnt not in key:
        processed_row = convert_data(line)
        processed_data.append(processed_row)
    cnt = cnt + 1
header = convert_data(lines[0])

<unknown>:1: SyntaxWarning: invalid decimal literal
<unknown>:1: SyntaxWarning: invalid decimal literal
<unknown>:1: SyntaxWarning: invalid decimal literal
<unknown>:1: SyntaxWarning: invalid decimal literal
<unknown>:1: SyntaxWarning: invalid decimal literal
<unknown>:1: SyntaxWarning: invalid decimal literal
<unknown>:1: SyntaxWarning: invalid decimal literal
<unknown>:1: SyntaxWarning: invalid decimal literal
<unknown>:1: SyntaxWarning: invalid decimal literal
<unknown>:1: SyntaxWarning: invalid decimal literal
<unknown>:1: SyntaxWarning: invalid decimal literal
<unknown>:1: SyntaxWarning: invalid decimal literal
<unknown>:1: SyntaxWarning: invalid decimal literal
<unknown>:1: SyntaxWarning: invalid decimal literal
<unknown>:1: SyntaxWarning: invalid decimal literal
<unknown>:1: SyntaxWarning: invalid decimal literal
<unknown>:1: SyntaxWarning: invalid decimal literal
<unknown>:1: SyntaxWarning: invalid decimal literal
<unknown>:1: SyntaxWarning: invalid decimal literal
<unknown>:1:

In [36]:
list_length_39 = []
list_length_38 = []
list_other_lengths = []

for sublist in processed_data:
    if len(sublist) == 39:
        list_length_39.append(sublist)
    elif len(sublist) == 38:
        list_length_38.append(sublist)
    else:
        list_other_lengths.append(sublist)
        
len(list_other_lengths), len(list_length_39) , len(list_length_38)

(228203, 137620, 134005)

In [37]:
for i in range(len(list_length_39)):
    if list_length_39[i][22] != 'nan':
        del list_length_39[i][23]

for sublist in list_length_39:
    assert len(sublist) == 38, f"List at index {list_length_39.index(sublist)} does not have length 38"

In [38]:
combined_list = list_length_39 + list_length_38
df_benign = pd.DataFrame(combined_list)
header[0] = 'Country_Name'
df_benign.columns = header
df_benign.replace('nan', np.nan, inplace=True)

In [39]:
df_benign = df_benign[column_order]

In [40]:
df_benign = df_benign[df_benign['puny_coded'].apply(lambda x: isinstance(x, int))]

In [41]:
df_benign['Domain_Age'] = df_benign['Domain_Age'].str.replace(' days', '')
df_benign['Domain_Age'] = pd.to_numeric(df_benign['Domain_Age'], errors='coerce').astype('Int64')

In [42]:
df_benign = df_benign[df_benign['oc_32'].apply(lambda x: isinstance(x, int) or pd.isnull(x))]
print(df_benign.shape)

df_benign = df_benign[df_benign['obfuscate_at_sign'].apply(lambda x: isinstance(x, int) or pd.isnull(x))]
print(df_benign.shape)


(271063, 38)
(271063, 38)


In [43]:
df_benign['oc_32'] = df_benign['oc_32'].astype(int)

In [44]:
df_benign['obfuscate_at_sign'] = df_benign['obfuscate_at_sign'].astype(int)

In [45]:
df_benign.info()

<class 'pandas.core.frame.DataFrame'>
Index: 271063 entries, 0 to 271624
Data columns (total 38 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Country_Name        211512 non-null  object 
 1   ASN                 211522 non-null  float64
 2   TTL                 271063 non-null  int64  
 3   IP                  212234 non-null  object 
 4   Domain              271063 non-null  object 
 5   oc_8                271063 non-null  object 
 6   longest_word        270946 non-null  object 
 7   Domain_Name         174709 non-null  object 
 8   dec_32              271063 non-null  object 
 9   subdomain           271063 non-null  float64
 10  sld                 270964 non-null  object 
 11  Registrant_Name     11378 non-null   object 
 12  hex_32              271063 non-null  float64
 13  Domain_Age          137039 non-null  Int64  
 14  entropy             271063 non-null  float64
 15  Name_Server_Count   217237 non-null  fl

In [46]:
df_benign.head()

,Country_Name,ASN,TTL,IP,Domain,oc_8,longest_word,Domain_Name,dec_32,subdomain,...,len,typos,tld,shortened,Registrar,obfuscate_at_sign,Alexa_Rank,3gram,numeric_percentage,puny_coded
0,NaN,NaN,38,NaN,b'www.google.com.',0,google,GOOGLE.COM,0,1.0,...,10,"[(google.com, 100), (google.com.br, 95)]",com,-1.0,MarkMonitor Inc.,0,1.0,"[goo, oog, ogl, gle]",0.0,0
1,NaN,NaN,95,NaN,b'youtube.com.',0,youtube,YOUTUBE.COM,0,0.0,...,8,"[(youtube.com, 100), (qq.com, 86)]",com,-1.0,MarkMonitor Inc.,0,2.0,"[you, out, utu, tub, ube]",0.0,0
2,US,1313.0,18,192.147.130.204,b'adobe.com.',0,adobe,ADOBE.COM,0,0.0,...,6,"[(adobe.com, 100), (qq.com, 86)]",com,-1.0,Nom-iq Ltd. dba COM LAUDE,0,55.0,"[ado, dob, obe]",0.0,0
3,NaN,NaN,729,NaN,b'www.adobe.com.',0,adobe,ADOBE.COM,0,1.0,...,9,"[(adobe.com, 100), (qq.com, 86)]",com,-1.0,Nom-iq Ltd. dba COM LAUDE,0,55.0,"[ado, dob, obe]",0.0,0
4,US,16625.0,729,104.107.49.85,b'www.adobe.com.',0,adobe,ADOBE.COM,0,1.0,...,9,"[(adobe.com, 100), (qq.com, 86)]",com,-1.0,Nom-iq Ltd. dba COM LAUDE,0,55.0,"[ado, dob, obe]",0.0,0


In [47]:
df_benign.to_csv('modified_benign.csv',index=False)

# Spam

In [48]:
with open("CSV_spam.csv", 'r') as file:
    lines = file.readlines()
header = convert_data(lines[0])

In [49]:
processed_data = []
for line in lines[1:]:
    processed_row = convert_data(line)
    processed_data.append(processed_row)

In [50]:
list_length_39 = []
list_length_38 = []
list_other_lengths = []

for sublist in processed_data:
    if len(sublist) == 39:
        list_length_39.append(sublist)
    elif len(sublist) == 38:
        list_length_38.append(sublist)
    else:
        list_other_lengths.append(sublist)
        
len(list_other_lengths), len(list_length_39) , len(list_length_38)

(1680, 827, 1830)

In [51]:
for i in range(len(list_length_39)):
    if type(list_length_39[i][-2]) == str and type(list_length_39[i][-3]) == str:
        del list_length_39[i][-2]

for sublist in list_length_39:
    assert len(sublist) == 38, f"List at index {list_length_39.index(sublist)} does not have length 38"

In [52]:
for i in range(len(list_other_lengths)):
    if type(list_other_lengths[i][-2]) == str and type(list_other_lengths[i][-3]) == str:
        del list_other_lengths[i][-2]

list_other_lengths = [sublist for sublist in list_other_lengths if len(sublist) != 38 or list_length_38.append(sublist)]

len(list_other_lengths), len(list_length_39) , len(list_length_38)

(1680, 827, 1830)

In [53]:
list_other_lengths[0]
cnt = 0
for i in list_other_lengths:
    if(len(i)) == 39:
        cnt = cnt+1
print(cnt)

1629


In [54]:
another_39_list = []
list_other_lengths = [sublist for sublist in list_other_lengths if len(sublist) != 39 or another_39_list.append(sublist)]
len(list_other_lengths), len(list_length_39) , len(list_length_38), len(another_39_list)

(51, 827, 1830, 1629)

In [55]:
another_39_list[0][16]

-1.0

In [56]:
for sublist in another_39_list:
    if isinstance(sublist[15], str) and isinstance(sublist[16], str):
        sublist[15] = sublist[15] + sublist[16]
        del sublist[16]
        

In [57]:
for sublist in another_39_list:
    if isinstance(sublist[18], str) and isinstance(sublist[19], str) and len(sublist) == 39:
        sublist[18] = sublist[18] + sublist[19]
        del sublist[19]

for sublist in another_39_list:
    assert len(sublist) == 38, f"List at index {list_length_39.index(sublist)} does not have length 38"

In [58]:
for sublist in list_other_lengths:
        sublist[15] = sublist[15] + sublist[16]
        sublist[19] = sublist[19] + sublist[20]
        del sublist[16]
        del sublist[19]

list_other_lengths = [sublist for sublist in list_other_lengths if len(sublist) != 38 or list_length_38.append(sublist)]

In [59]:
print(len(list_other_lengths), len(list_length_39) , len(list_length_38), len(another_39_list))
for sublist in list_length_38:
    assert len(sublist) == 38, f"List at index {list_length_39.index(sublist)} does not have length 38"
for sublist in list_length_39:
    assert len(sublist) == 38, f"List at index {list_length_39.index(sublist)} does not have length 38"
for sublist in another_39_list:
    assert len(sublist) == 38, f"List at index {list_length_39.index(sublist)} does not have length 38"

0 827 1881 1629


In [60]:
print(len(list_other_lengths), len(list_length_39) , len(list_length_38), len(another_39_list))

0 827 1881 1629


In [61]:
combined_list = list_length_39 + list_length_38 + another_39_list
df_spam = pd.DataFrame(combined_list)
header[0] = 'Country_Name'
df_spam.columns = header
df_spam.replace('nan', np.nan, inplace=True)
df_spam = df_spam[column_order]

In [62]:
df_spam['Page_Rank'][10]

-1.0

In [63]:
non_int_indices = df_spam.index[
    ~df_spam['Page_Rank'].apply(lambda x: isinstance(x, float)) & df_spam['Page_Rank'].notna()
]
non_int_indices


Index([], dtype='int64')

In [64]:
df_spam['Domain_Age'] = df_spam['Domain_Age'].str.replace(' days', '')
df_spam['Domain_Age'] = pd.to_numeric(df_spam['Domain_Age'], errors='coerce').astype('Int64')

In [65]:
df_spam.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4337 entries, 0 to 4336
Data columns (total 38 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Country_Name        3490 non-null   object 
 1   ASN                 3490 non-null   float64
 2   TTL                 4337 non-null   int64  
 3   IP                  3587 non-null   object 
 4   Domain              4337 non-null   object 
 5   oc_8                4337 non-null   int64  
 6   longest_word        4337 non-null   object 
 7   Domain_Name         2627 non-null   object 
 8   dec_32              4337 non-null   int64  
 9   subdomain           4337 non-null   int64  
 10  sld                 4337 non-null   object 
 11  Registrant_Name     20 non-null     object 
 12  hex_32              4337 non-null   int64  
 13  Domain_Age          2507 non-null   Int64  
 14  entropy             4337 non-null   float64
 15  Name_Server_Count   3145 non-null   float64
 16  Emails

In [66]:
df_spam.head()

,Country_Name,ASN,TTL,IP,Domain,oc_8,longest_word,Domain_Name,dec_32,subdomain,...,len,typos,tld,shortened,Registrar,obfuscate_at_sign,Alexa_Rank,3gram,numeric_percentage,puny_coded
0,US,22612.0,1,198.54.112.216,b'5805.scenbe.com.',0,be,SCENBE.COM,0,1,...,11,"[(qq.com, 86), (vk.com, 86)]",com,-1,UNIREGISTRAR CORP,0,-1.0,"[sce, cen, enb, nbe]",23.529412,0
1,EE,202759.0,599,185.155.96.63,b'allyncoralyn.ru.',0,coral,ALLYNCORALYN.RU,0,0,...,13,"[(yandex.ru, 86), (mail.ru, 86)]",ru,-1,R01-RU,0,-1.0,"[all, lly, lyn, ync, nco, cor, ora, ral, aly, ...",0.000000,0
2,RU,8342.0,10799,195.161.41.33,b'alrayedtravel.com.',0,travel,ALRAYEDTRAVEL.COM,0,0,...,14,"[(twitter.com, 86), (google.com, 86)]",com,-1,Registrar of domain names REG.RU LLC,0,-1.0,"[alr, lra, ray, aye, yed, edt, dtr, tra, rav, ...",0.000000,0
3,EE,202759.0,599,185.155.96.63,b'annaliesefayre.ru.',0,liese,ANNALIESEFAYRE.RU,0,0,...,15,"[(yandex.ru, 86), (mail.ru, 86)]",ru,-1,R01-RU,0,-1.0,"[ann, nna, nal, ali, lie, ies, ese, sef, efa, ...",0.000000,0
4,US,33387.0,21599,192.151.158.114,b'avisosindustriales.me.',0,industrial,AVISOSINDUSTRIALES.ME,0,0,...,19,"[(line.me, 86), (wp.me, 86)]",me,-1,Instra Corporation Pty Ltd.,0,-1.0,"[avi, vis, iso, sos, osi, sin, ind, ndu, dus, ...",0.000000,0


In [67]:
df_spam.to_csv('modified_spam.csv',index=False)

# Phishing

In [68]:
with open("CSV_phishing.csv", 'r') as file:
    lines = file.readlines()
header = convert_data(lines[0])

In [69]:
processed_data = []
for line in lines[1:]:
    processed_row = convert_data(line)
    processed_data.append(processed_row)

In [70]:
list_length_39 = []
list_length_38 = []
list_other_lengths = []

for sublist in processed_data:
    if len(sublist) == 39:
        list_length_39.append(sublist)
    elif len(sublist) == 38:
        list_length_38.append(sublist)
    else:
        list_other_lengths.append(sublist)
        
len(list_other_lengths), len(list_length_39) , len(list_length_38)

(2029, 1188, 1784)

In [71]:
i_in_39 = [953, 954, 1011, 1012, 1013, 1014, 1015]

In [72]:
for x in i_in_39:
    list_length_39[x][27] = list_length_39[x][27] + list_length_39[x][28]
    del list_length_39[x][28]
    

In [73]:
list_length_39 = [sublist for sublist in list_length_39 if len(sublist) == 39 or list_length_38.append(sublist)]
len(list_other_lengths), len(list_length_39) , len(list_length_38)

(2029, 1181, 1791)

In [74]:
for i in range(len(list_length_39)):
    if(list_length_39[i][6] != 'nan'):
        del list_length_39[i][7]

for sublist in list_length_39:
    assert len(sublist) == 38, f"List at index {list_length_39.index(sublist)} does not have length 38"

In [75]:
for i in range(len(list_other_lengths)):
    if(list_other_lengths[i][6] != 'nan'):
        del list_other_lengths[i][7]

In [76]:
len(list_other_lengths), len(list_length_39) , len(list_length_38)

(2029, 1181, 1791)

In [77]:
another_39_list = []
list_other_lengths = [sublist for sublist in list_other_lengths if len(sublist) != 39 or another_39_list.append(sublist)]
len(list_other_lengths), len(list_length_39) , len(list_length_38), len(another_39_list)

(367, 1181, 1791, 1662)

In [78]:
id = [1, 2, 9, 10, 21, 27, 28, 30, 31, 40, 41, 42, 43, 44, 76, 77, 78, 79, 92, 104, 105, 106, 107, 109, 110, 111, 112, 113, 
      115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 
      138, 139, 140, 141, 142, 143, 144, 195, 216, 225, 226, 227, 229, 230, 233, 234, 262, 263, 266, 348, 349, 351, 352,
      372, 394, 395, 396, 397, 398, 399, 400, 401, 403, 404, 418, 420, 421, 430, 431, 445, 449, 487, 501, 503, 507, 508,
      512, 517, 522, 523, 529, 530, 537, 570, 596, 597, 630, 634, 639, 677, 678, 697, 698, 699, 700, 701, 702, 704, 708, 709,
      714, 771, 772, 773, 774, 775, 777, 778, 779, 781, 785, 787, 790, 793, 794, 801, 808, 809, 818, 819, 830, 835, 836, 838, 
      839, 840, 842, 844, 845, 847, 890, 926, 927, 928, 930, 931, 937, 944, 967, 981, 982, 983, 1018, 1019, 1020, 1021, 1030,
      1031, 1037, 1038, 1045, 1046, 1047, 1048, 1049, 1051, 1052, 1053, 1056, 1064, 1066, 1091, 1106, 1111, 1114, 1115, 1116,
      1118, 1152, 1156, 1157, 1159, 1160, 1161, 1162, 1164, 1186, 1226,1228, 1229, 1290, 1320, 1321, 1325, 1333, 1360, 1473, 1499, 
      1507, 1508, 1529, 1536, 1541, 1573, 1578, 1579, 1641]


In [79]:
for i in sorted(id, reverse=True):
    another_39_list[i][16] = another_39_list[i][16] + another_39_list[i][17]  
    del another_39_list[i][17]  
    list_length_38.append(another_39_list[i])  
    del another_39_list[i]  

In [80]:
len(list_other_lengths), len(list_length_39) , len(list_length_38), len(another_39_list)

(367, 1181, 2009, 1444)

In [81]:
cnt = 0
for sublist in another_39_list:
   
    if isinstance(sublist[16], str) and isinstance(sublist[17], str) and isinstance(sublist[18], str) and isinstance(sublist[19], str) and isinstance(sublist[20], int):
   
        cnt = cnt + 1
        sublist[16] = sublist[16] + sublist[17]
        del sublist[17]
print(cnt)

56


In [82]:
another_39_list = [sublist for sublist in another_39_list if len(sublist) != 38 or list_length_38.append(sublist)]
len(list_other_lengths), len(list_length_39) , len(list_length_38), len(another_39_list)

(367, 1181, 2065, 1388)

In [83]:
for sublist in another_39_list:
    # print(sublist[27],sublist[28],sublist[27]
    if isinstance(sublist[27], str) and isinstance(sublist[28], str) and isinstance(sublist[29], int):
        # print("k")
        sublist[27] = sublist[27] + sublist[28]
        del sublist[28]

another_39_list = [sublist for sublist in another_39_list if len(sublist) != 38 or list_length_38.append(sublist)]
len(list_other_lengths), len(list_length_39) , len(list_length_38), len(another_39_list)

(367, 1181, 3429, 24)

In [84]:
for sublist in another_39_list:
    # print(sublist[27],sublist[28],sublist[27]
    if sublist[27] == 75500:
        # print("k")
        k = str(sublist[27])
        sublist[26] = sublist[26] + k
        # print(sublist[26])
        del sublist[27]
another_39_list = [sublist for sublist in another_39_list if len(sublist) != 38 or list_length_38.append(sublist)]
len(list_other_lengths), len(list_length_39) , len(list_length_38), len(another_39_list)

(367, 1181, 3453, 0)

In [85]:
cnt1 = 0
cnt2 = 0
for i in list_other_lengths:
    if(len(i) == 40):
        cnt1 = cnt1 + 1
    elif(len(i)==41):
        cnt2 = cnt2 + 1
cnt1,cnt2

(367, 0)

In [86]:
for sublist in list_other_lengths:
        sublist[16] = sublist[16] + sublist[17]
        sublist[28] = sublist[28] + sublist[29]
        del sublist[17]
        del sublist[28]

list_other_lengths = [sublist for sublist in list_other_lengths if len(sublist) != 38 or list_length_38.append(sublist)]
len(list_other_lengths), len(list_length_39) , len(list_length_38), len(another_39_list)

(0, 1181, 3820, 0)

In [87]:
combined_list = list_length_38 + list_length_39

In [88]:
combined_list
df_phishing = pd.DataFrame(combined_list)
header[0] = 'Country_Name'
df_phishing.columns = header
df_phishing.replace('nan', np.nan, inplace=True)
df_phishing = df_phishing[column_order]

In [89]:
ind = [3477, 3597, 3598, 3601, 3626, 3627, 3632, 3633, 3638, 3644, 3645, 3664,
       3668, 3669, 3670, 3671, 3672, 3673, 3674, 3675, 3676, 3677, 3678, 3679,
       3758, 3759, 1826, 4047, 4048, 4428, 4487, 4546,4782]

In [90]:
ind = [3477, 3597, 3598, 3601, 3626, 3627, 3632, 3633, 3638, 3644, 3645, 3664,
       3668, 3669, 3670, 3671, 3672, 3673, 3674, 3675, 3676, 3677, 3678, 3679,
       3758, 3759, 1826, 4047, 4048, 4428, 4487, 4546,4782]
df_phishing = df_phishing.drop(ind, errors='ignore')


In [91]:
non_int_indices = df_phishing.index[
    ~df_phishing['dec_32'].apply(lambda x: isinstance(x, int)) & df_phishing['dec_32'].notna()
]
non_int_indices



Index([], dtype='int64')

In [92]:
df_phishing['Domain_Age'] = df_phishing['Domain_Age'].str.replace(' days', '')
df_phishing['Domain_Age'] = pd.to_numeric(df_phishing['Domain_Age'], errors='coerce').astype('Int64')

In [93]:
df_phishing = df_phishing[df_phishing['dec_32'].apply(lambda x: isinstance(x, int) or pd.isnull(x))]
print(df_phishing.shape)


(4968, 38)


In [94]:
non_int_rows = df_phishing[
    ~df_phishing['oc_8'].apply(lambda x: isinstance(x, int)) & df_phishing['oc_8'].notna()
]
non_int_rows

,Country_Name,ASN,TTL,IP,Domain,oc_8,longest_word,Domain_Name,dec_32,subdomain,...,len,typos,tld,shortened,Registrar,obfuscate_at_sign,Alexa_Rank,3gram,numeric_percentage,puny_coded


In [95]:
df_phishing['oc_8'] = df_phishing['oc_8'].astype(int)

In [96]:
df_phishing['dec_32'] = df_phishing['dec_32'].astype(int)

In [97]:
df_phishing['obfuscate_at_sign'] = df_phishing['obfuscate_at_sign'].astype(int)

In [98]:
df_phishing['puny_coded'] = df_phishing['puny_coded'].astype(float)

In [99]:
df_phishing.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4968 entries, 0 to 5000
Data columns (total 38 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Country_Name        3556 non-null   object 
 1   ASN                 3556 non-null   float64
 2   TTL                 4968 non-null   int64  
 3   IP                  3566 non-null   object 
 4   Domain              4968 non-null   object 
 5   oc_8                4968 non-null   int64  
 6   longest_word        4968 non-null   object 
 7   Domain_Name         3337 non-null   object 
 8   dec_32              4968 non-null   int64  
 9   subdomain           4968 non-null   int64  
 10  sld                 4968 non-null   object 
 11  Registrant_Name     34 non-null     object 
 12  hex_32              4968 non-null   int64  
 13  Domain_Age          3177 non-null   Int64  
 14  entropy             4968 non-null   float64
 15  Name_Server_Count   3934 non-null   float64
 16  Emails     

In [100]:
df_phishing.head()

,Country_Name,ASN,TTL,IP,Domain,oc_8,longest_word,Domain_Name,dec_32,subdomain,...,len,typos,tld,shortened,Registrar,obfuscate_at_sign,Alexa_Rank,3gram,numeric_percentage,puny_coded
0,NL,204915.0,2,145.14.145.41,b'programafidelidadeitacard2.cf.',0,program,NaN,0,0,...,27,"[(deviantart.com, 53), (canada.ca, 53)]",cf,-1.0,NaN,0,-1.0,"[pro, rog, ogr, gra, ram, ama, maf, afi, fid, ...",3.225806,0.0
1,NaN,NaN,31,NaN,b'programafidelidadeitacard2.cf.',0,program,NaN,0,0,...,27,"[(deviantart.com, 53), (canada.ca, 53)]",cf,-1.0,NaN,0,-1.0,"[pro, rog, ogr, gra, ram, ama, maf, afi, fid, ...",3.225806,0.0
2,NL,52000.0,199,91.224.140.71,b'gg.gg.',0,gg,NaN,0,0,...,3,"[(blogger.com, 86), (digg.com, 86)]",gg,-1.0,NaN,0,44394.0,[],0.0,0.0
3,US,14618.0,59,52.71.139.107,b'split.to.',0,split,NaN,0,0,...,6,"[(t.co, 68), (slate.com, 59)]",to,-1.0,NaN,0,1801349.0,"[spl, pli, lit]",0.0,0.0
4,NaN,NaN,59,NaN,b'split.to.',0,split,NaN,0,0,...,6,"[(t.co, 68), (slate.com, 59)]",to,-1.0,NaN,0,1801349.0,"[spl, pli, lit]",0.0,0.0


In [101]:
df_phishing.to_csv('modified_phishing.csv',index=False)